In [1]:
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import math

**DATA PREPROCESSING**

In [2]:
for i in range(1, 16):
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/data_num/ds'+ str(i) +'.csv')
    print('='*50)
    print('shape of {}:'.format(i), df.shape)
    print(df.columns)
    if df.isnull().sum().sum() != 0:
        print(df.isnull().sum())
        break

shape of 1: (250, 13)
Index(['surgery', 'Age', 'Hospital_Number', 'pulse', 'mucous_membranes',
       'capillary_refill_time', 'peristalsis', 'packed_cell_volume',
       'total_protein', 'outcome', 'surgical_lesion', 'site_of_lesion',
       'pathology_cp_data'],
      dtype='object')
shape of 2: (250, 10)
Index(['surgery', 'Age', 'pulse', 'mucous_membranes', 'capillary_refill_time',
       'peristalsis', 'packed_cell_volume', 'total_protein', 'outcome',
       'surgical_lesion'],
      dtype='object')
shape of 3: (306, 4)
Index(['Age_of_patient_at_time_of_operation', 'Patients_year_of_operation',
       'Number_of_positive_axillary_nodes_detected', 'Survival_status'],
      dtype='object')
shape of 4: (310, 7)
Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'Class'], dtype='object')
shape of 5: (320, 7)
Index(['ACTION', 'CONDITIONAL', 'CONTINUANCE', 'IMPERATIVE', 'RISK_LEVEL',
       'SOURCE', 'DEFECT'],
      dtype='object')
shape of 6: (327, 38)
Index(['a', 'b', 'c', 'd', 'e', 'f', 'g',

**EXPERIMENT DEFINITION**

In [3]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt
from sklearn import metrics
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras import optimizers



def predict(result, X_test):
    predicted = []
    for j in range(X_test.shape[0]):
        if result[j] <= threshold:
            predicted.append(0)
        else:
            predicted.append(1)
    return predicted

def predict_ours(result, X_test):
    predicted = []
    for j in range(X_test.shape[0]):
        if result[j] <= threshold:
            predicted.append(0)
        else:
            predicted.append(1)
    return predicted

def MSE(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true - y_pred))

def BCE(y_true, y_pred):
    return -tf.reduce_mean(y_true*tf.math.log(y_pred)+(1-y_true)*tf.math.log(1-y_pred))

def Ours_Accu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    yl = y_train.shape[0]
    accu = (yl-tf.reduce_sum(y_true)-tf.reduce_sum(y_pred)+2*tf.reduce_sum(y_true*y_pred)) / yl
    return 1-accu

def Ours_Fbeta(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    numerator = (1+beta**2)*tf.reduce_sum(y_true*y_pred)
    denominator = (beta**2)*tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1-(numerator/denominator)

def Ours_Gmean(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    syhy = tf.reduce_sum(y_true*y_pred)
    sy = tf.reduce_sum(y_true)
    yl = y_train.shape[0]
    gmean = tf.sqrt(syhy*(yl-tf.reduce_sum(y_pred)-sy+syhy)/(sy*(yl-sy)))
    return 1-gmean

def Ours_BAccu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    syhy = tf.reduce_sum(y_true*y_pred)
    sy = tf.reduce_sum(y_true)
    yl = y_train.shape[0]
    baccu = (yl*(syhy+sy)-sy*(tf.reduce_sum(y_pred)+sy)) / (2*sy*(yl-sy))
    return 1-baccu

def get_results(y, predicted):
    TN = metrics.confusion_matrix(y, predicted)[0,0]
    FP = metrics.confusion_matrix(y, predicted)[0,1]
    FN = metrics.confusion_matrix(y, predicted)[1,0]
    TP = metrics.confusion_matrix(y, predicted)[1,1]
    acc = (TP+TN)/(TP+TN+FP+FN)
    f1 = TP/(TP + 0.5*(FP+FN))
    f05 = TP/(TP + 0.8*FP + 0.2*FN)
    f2 = TP/(TP + 0.2*FP + 0.8*FN)
    gmean = ((TP/(TP+FN)) * (TN/(TN+FP)))**0.5
    bacc = 0.5*(TP/(TP+FN) + TN/(TN+FP))

    return np.round(acc,4), np.round(f1,4), np.round(f05,4), np.round(f2,4), np.round(gmean,4), np.round(bacc,4)


**15 DATASETS METADATA**

In [4]:
data = []
sample = []
feature = []
imbalance1 = []
imbalance2 = []

for i in range(1, 16):
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/data_num/ds'+ str(i) +'.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    print('<Original Class>\n', df.iloc[:,-1].value_counts())

    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0]
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0]
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "{: .2f}:1".format(df.iloc[:,-1].value_counts()[0]/df.iloc[:,-1].value_counts()[1]))
    data.append(i)
    imbalance1.append(df.iloc[:,-1].value_counts()[0]/len(df))
    imbalance2.append(df.iloc[:,-1].value_counts()[1]/len(df))

    X = df.iloc[:, :-1]
    X = (X - X.mean())/X.std()
    y = df.iloc[:, -1]
    sample.append(X.shape[0])
    feature.append(X.shape[1])

+++++++++++++++++++++++++++++++++++ 1th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 pathology_cp_data
2    160
1     90
Name: count, dtype: int64
<Modified Class>
 pathology_cp_data
0    160
1     90
Name: count, dtype: int64
<Imabalance ratio>
  1.78:1
+++++++++++++++++++++++++++++++++++ 2th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 surgical_lesion
2    154
1     96
Name: count, dtype: int64
<Modified Class>
 surgical_lesion
0    154
1     96
Name: count, dtype: int64
<Imabalance ratio>
  1.60:1
+++++++++++++++++++++++++++++++++++ 3th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 Survival_status
1    225
2     81
Name: count, dtype: int64
<Modified Class>
 Survival_status
0    225
1     81
Name: count, dtype: int64
<Imabalance ratio>
  2.78:1
+++++++++++++++++++++++++++++++++++ 4th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 Class
1    210
2    100
Name: count, dtype: int64
<Modified Class>
 Class
0    210
1    10

In [5]:
print("Data:")
print(data)

print("No. of samples: ")
print(sample)

print("No. of features: ")
print(feature)

print("Imbalance ratio 1: ")
print(imbalance1)

print("Imbalance ratio 2:")
print(imbalance2)


Data:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
No. of samples: 
[250, 250, 306, 310, 320, 327, 328, 335, 336, 349, 351, 358, 363, 365, 381]
No. of features: 
[12, 9, 3, 6, 6, 37, 32, 3, 14, 31, 33, 31, 8, 5, 38]
Imbalance ratio 1: 
[np.float64(0.64), np.float64(0.616), np.float64(0.7352941176470589), np.float64(0.6774193548387096), np.float64(0.665625), np.float64(0.8715596330275229), np.float64(0.6890243902439024), np.float64(0.8537313432835821), np.float64(0.7559523809523809), np.float64(0.6217765042979942), np.float64(0.6410256410256411), np.float64(0.6899441340782123), np.float64(0.7658402203856749), np.float64(0.915068493150685), np.float64(0.847769028871391)]
Imbalance ratio 2:
[np.float64(0.36), np.float64(0.384), np.float64(0.2647058823529412), np.float64(0.3225806451612903), np.float64(0.334375), np.float64(0.12844036697247707), np.float64(0.31097560975609756), np.float64(0.14626865671641792), np.float64(0.24404761904761904), np.float64(0.37822349570200575), np.floa

In [6]:
pd.options.display.max_rows = 20
df_im = pd.DataFrame(data, columns=["data#"] )
df_im["sample"] = sample
df_im["feature"] = feature
df_im["negative(0)"] = imbalance1
df_im["positive(1)"] = imbalance2
df_im

,data#,sample,feature,negative(0),positive(1)
0,1,250,12,0.640000,0.360000
1,2,250,9,0.616000,0.384000
2,3,306,3,0.735294,0.264706
3,4,310,6,0.677419,0.322581
4,5,320,6,0.665625,0.334375
5,6,327,37,0.871560,0.128440
6,7,328,32,0.689024,0.310976
7,8,335,3,0.853731,0.146269
8,9,336,14,0.755952,0.244048
9,10,349,31,0.621777,0.378223


In [11]:
data = []
sample = []
feature = []
imbalance1 = []
imbalance2 = []

from keras import initializers
L = 73
hidden_node = 2
activation = 'sigmoid'
kernel_initializer=keras.initializers.he_normal(seed=50)
epochs=10
threshold = 0.6

res = pd.DataFrame({'Dataset':['D', 'S', 0, 'C', 'S', 'V'],
                    'Delta':[0, 0, 0, 0, 0, 0],
                    'BCE':[0, 0, 0, 0, 0, 0],
                    'AccAim':[0, 0, 0, 0, 0, 0],
                    'F1Aim':[0, 0, 0, 0, 0, 0],
                    'F.5Aim':[0, 0, 0, 0, 0, 0],
                    'F2Aim':[0, 0, 0, 0, 0, 0],
                    'GAim':[0, 0, 0, 0, 0, 0],
                    'BAim':[0, 0, 0, 0, 0, 0]}, index = ['Acc','F1','F.5','F2','Gmean','B_Acc'])

for i in range(1, 16):
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Major-project-8th-sem/data_num/ds'+ str(i) +'.csv')
    print('+'*35, '{}th Dataset'.format(i), '+'*35)
    print('<Original Class>\n', df.iloc[:,-1].value_counts())
    MAJOR = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() == max(df.iloc[:,-1].value_counts())].index[0]
    minor = df.iloc[:,-1].value_counts()[df.iloc[:,-1].value_counts() != max(df.iloc[:,-1].value_counts())].index[0]
    df.iloc[:,-1] = df.iloc[:,-1].replace(MAJOR, -100)
    df.iloc[:,-1] = df.iloc[:,-1].replace(minor, 1)
    df.iloc[:,-1] = df.iloc[:,-1].replace(-100, 0)
    print('<Modified Class>\n', df.iloc[:,-1].value_counts())
    print('<Imabalance ratio>\n', "{: .2f}:1".format(df.iloc[:,-1].value_counts()[0]/df.iloc[:,-1].value_counts()[1]))

    X = df.iloc[:, :-1]
    X = (X - X.mean())/X.std()
    y = df.iloc[:, -1]


    ada_acc = []
    ada_f1 = []
    ada_f5 = []
    ada_f2 = []
    ada_gmean = []
    ada_bacc = []

    bce_acc = []
    bce_f1 = []
    bce_f5 = []
    bce_f2 = []
    bce_gmean = []
    bce_bacc = []

    acc_acc = []
    acc_f1 = []
    acc_f5 = []
    acc_f2 = []
    acc_gmean = []
    acc_bacc = []

    f1_acc = []
    f1_f1 = []
    f1_f5 = []
    f1_f2 = []
    f1_gmean = []
    f1_bacc = []

    f05_acc = []
    f05_f1 = []
    f05_f5 = []
    f05_f2 = []
    f05_gmean = []
    f05_bacc = []

    f2_acc = []
    f2_f1 = []
    f2_f5 = []
    f2_f2 = []
    f2_gmean = []
    f2_bacc = []

    gmean_acc = []
    gmean_f1 = []
    gmean_f5 = []
    gmean_f2 = []
    gmean_gmean = []
    gmean_bacc = []

    bacc_acc = []
    bacc_f1 = []
    bacc_f5 = []
    bacc_f2 = []
    bacc_gmean = []
    bacc_bacc = []

    model_MSE = Sequential()
    model_MSE.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
    model_MSE.add(BatchNormalization())
    model_MSE.add(Activation(activation))
    model_MSE.add(Dense(1))

    model = Sequential()
    model.add(Dense(hidden_node, input_dim=X.shape[1], kernel_initializer=kernel_initializer))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dense(1, activation='sigmoid'))

    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state = 2)
    n_iter=0

    for train_index, test_index in skf.split(df, df.iloc[:,-1]):
        n_iter += 1
        X_train = X.iloc[train_index]
        y_train= y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test= y.iloc[test_index]
        print('#'*50,'{0}th CV'.format(n_iter),'#'*50)

        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)

        print('<Class Distribution>\n',
              'Train 0:1\n'.format(i), list(y_train).count(0), list(y_train).count(1), '\n',
              'Test 0:1\n'.format(i), list(y_test).count(0), list(y_test).count(1))

        batch_size = int(X_train.shape[0] * 0.05)
        learning_rate = 0.005

        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model_MSE.compile(loss=MSE, optimizer=opt, metrics=['accuracy'])
        history = model_MSE.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)

        print("------------"*2, "Delta Rule", "------------"*2)
        result = model_MSE.predict(X_test)
        predicted = np.round(result)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        ada_acc.append(acc)
        ada_f1.append(f1)
        ada_f5.append(f05)
        ada_f2.append(f2)
        ada_gmean.append(gmean)
        ada_bacc.append(bacc)


        batch_size = int(X_train.shape[0] * 0.05)
        learning_rate=0.003

        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=BCE, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)

        print("------------"*2, "Logistic Regression", "------------"*2)
        result = model.predict(X_test)
        predicted = np.round(result)

        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        bce_acc.append(acc)
        bce_f1.append(f1)
        bce_f5.append(f05)
        bce_f2.append(f2)
        bce_gmean.append(gmean)
        bce_bacc.append(bacc)

        batch_size = int(X_train.shape[0] * 0.05)
        learning_rate = 0.001

        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=Ours_Accu, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)

        print("------------"*2, "Our Method (Accuracy)", "------------"*2)
        result = 1/(1+tf.math.exp(-L*(model.predict(X_test)-0.5)))
        predicted = np.round(result)


        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        acc_acc.append(acc)
        acc_f1.append(f1)
        acc_f5.append(f05)
        acc_f2.append(f2)
        acc_gmean.append(gmean)
        acc_bacc.append(bacc)


        batch_size = int(X_train.shape[0] * 0.05)
        learning_rate=0.001

        beta = 1
        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=Ours_Fbeta, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)



        print("------------"*2, "Our Method (F1)", "------------"*2)
        result = 1/(1+tf.math.exp(-L*(model.predict(X_test)-0.5)))
        predicted = np.round(result)
        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        f1_acc.append(acc)
        f1_f1.append(f1)
        f1_f5.append(f05)
        f1_f2.append(f2)
        f1_gmean.append(gmean)
        f1_bacc.append(bacc)
        batch_size = int(X_train.shape[0] * 0.05)
        learning_rate=0.001

        beta = 0.5
        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=Ours_Fbeta, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)

        print("------------"*2, "Our Method (F0.5)", "------------"*2)
        result = 1/(1+tf.math.exp(-L*(model.predict(X_test)-0.5)))
        predicted = np.round(result)


        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        f05_acc.append(acc)
        f05_f1.append(f1)
        f05_f5.append(f05)
        f05_f2.append(f2)
        f05_gmean.append(gmean)
        f05_bacc.append(bacc)

        batch_size = int(X_train.shape[0] * 0.05)
        learning_rate=0.001

        beta = 2
        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=Ours_Fbeta, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)


        print("------------"*2, "Our Method (F2)", "------------"*2)
        result = 1/(1+tf.math.exp(-L*(model.predict(X_test)-0.5)))
        predicted = np.round(result)

        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        f2_acc.append(acc)
        f2_f1.append(f1)
        f2_f5.append(f05)
        f2_f2.append(f2)
        f2_gmean.append(gmean)
        f2_bacc.append(bacc)

        batch_size = int(np.ceil(X.shape[0]*0.9 * 0.5))
        learning_rate=0.001

        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=Ours_Gmean, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch_size)

        print("------------"*2, "Our Method (Gmean)", "------------"*2)
        result = 1/(1+tf.math.exp(-L*(model.predict(X_test)-0.5)))
        predicted = np.round(result)


        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        gmean_acc.append(acc)
        gmean_f1.append(f1)
        gmean_f5.append(f05)
        gmean_f2.append(f2)
        gmean_gmean.append(gmean)
        gmean_bacc.append(bacc)
        batch_size = int(X_train.shape[0] * 0.5)
        learning_rate=0.001

        opt = optimizers.Adam(learning_rate = learning_rate)   # SGD(learning_rate=learning_rate, momentum=momentum)
        model.compile(loss=Ours_BAccu, optimizer=opt, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=X_train.shape[0])

        print("------------"*2, "Our Method (B_Accuracy)", "------------"*2)
        result = 1/(1+tf.math.exp(-L*(model.predict(X_test)-0.5)))
        predicted = np.round(result)

        acc, f1, f05, f2, gmean, bacc = get_results(y_test, predicted)
        bacc_acc.append(acc)
        bacc_f1.append(f1)
        bacc_f5.append(f05)
        bacc_f2.append(f2)
        bacc_gmean.append(gmean)
        bacc_bacc.append(bacc)

    res.iloc[2,0] = i

    final_ada_acc = np.nanmean(ada_acc)
    res.iloc[0,1] = final_ada_acc
    final_ada_f1 = np.nanmean(ada_f1)
    res.iloc[1,1] = final_ada_f1
    final_ada_f5 = np.nanmean(ada_f5)
    res.iloc[2,1] = final_ada_f5
    final_ada_f2 = np.nanmean(ada_f2)
    res.iloc[3,1] = final_ada_f2
    final_ada_gmean = np.nanmean(ada_gmean)
    res.iloc[4,1] = final_ada_gmean
    final_ada_bacc = np.nanmean(ada_bacc)
    res.iloc[5,1] = final_ada_bacc

    final_bce_acc = np.nanmean(bce_acc)
    res.iloc[0,2] = final_bce_acc
    final_bce_f1 = np.nanmean(bce_f1)
    res.iloc[1,2] = final_bce_f1
    final_bce_f5 = np.nanmean(bce_f5)
    res.iloc[2,2] = final_bce_f5
    final_bce_f2 = np.nanmean(bce_f2)
    res.iloc[3,2] = final_bce_f2
    final_bce_gmean = np.nanmean(bce_gmean)
    res.iloc[4,2] = final_bce_gmean
    final_bce_bacc = np.nanmean(bce_bacc)
    res.iloc[5,2] = final_bce_bacc

    final_acc_acc = np.nanmean(acc_acc)
    res.iloc[0,3] = final_acc_acc
    final_acc_f1 = np.nanmean(acc_f1)
    res.iloc[1,3] = final_acc_f1
    final_acc_f5 = np.nanmean(acc_f5)
    res.iloc[2,3] = final_acc_f5
    final_acc_f2 = np.nanmean(acc_f2)
    res.iloc[3,3] = final_acc_f2
    final_acc_gmean = np.nanmean(acc_gmean)
    res.iloc[4,3] = final_acc_gmean
    final_acc_bacc = np.nanmean(acc_bacc)
    res.iloc[5,3] = final_acc_bacc

    final_f1_acc = np.nanmean(f1_acc)
    res.iloc[0,4] = final_f1_acc
    final_f1_f1 = np.nanmean(f1_f1)
    res.iloc[1,4] = final_f1_f1
    final_f1_f5 = np.nanmean(f1_f5)
    res.iloc[2,4] = final_f1_f5
    final_f1_f2 = np.nanmean(f1_f2)
    res.iloc[3,4] = final_f1_f2
    final_f1_gmean = np.nanmean(f1_gmean)
    res.iloc[4,4] = final_f1_gmean
    final_f1_bacc = np.nanmean(f1_bacc)
    res.iloc[5,4] = final_f1_bacc

    final_f05_acc = np.nanmean(f05_acc)
    res.iloc[0,5] = final_f05_acc
    final_f05_f1 = np.nanmean(f05_f1)
    res.iloc[1,5] = final_f05_f1
    final_f05_f5 = np.nanmean(f05_f5)
    res.iloc[2,5] = final_f05_f5
    final_f05_f2 = np.nanmean(f05_f2)
    res.iloc[3,5] = final_f05_f2
    final_f05_gmean = np.nanmean(f05_gmean)
    res.iloc[4,5] = final_f05_gmean
    final_f05_bacc = np.nanmean(f05_bacc)
    res.iloc[5,5] = final_f05_bacc

    final_f2_acc = np.nanmean(f2_acc)
    res.iloc[0,6] = final_f2_acc
    final_f2_f1 = np.nanmean(f2_f1)
    res.iloc[1,6] = final_f2_f1
    final_f2_f5 = np.nanmean(f2_f5)
    res.iloc[2,6] = final_f2_f5
    final_f2_f2 = np.nanmean(f2_f2)
    res.iloc[3,6] = final_f2_f2
    final_f2_gmean = np.nanmean(f2_gmean)
    res.iloc[4,6] = final_f2_gmean
    final_f2_bacc = np.nanmean(f2_bacc)
    res.iloc[5,6] = final_f2_bacc

    final_gmean_acc = np.nanmean(gmean_acc)
    res.iloc[0,7] = final_gmean_acc
    final_gmean_f1 = np.nanmean(gmean_f1)
    res.iloc[1,7] = final_gmean_f1
    final_gmean_f5 = np.nanmean(gmean_f5)
    res.iloc[2,7] = final_gmean_f5
    final_gmean_f2 = np.nanmean(gmean_f2)
    res.iloc[3,7] = final_gmean_f2
    final_gmean_gmean = np.nanmean(gmean_gmean)
    res.iloc[4,7] = final_gmean_gmean
    final_gmean_bacc = np.nanmean(gmean_bacc)
    res.iloc[5,7] = final_gmean_bacc

    final_bacc_acc = np.nanmean(bacc_acc)
    res.iloc[0,8] = final_bacc_acc
    final_bacc_f1 = np.nanmean(bacc_f1)
    res.iloc[1,8] = final_bacc_f1
    final_bacc_f5 = np.nanmean(bacc_f5)
    res.iloc[2,8] = final_bacc_f5
    final_bacc_f2 = np.nanmean(bacc_f2)
    res.iloc[3,8] = final_bacc_f2
    final_bacc_gmean = np.nanmean(bacc_gmean)
    res.iloc[4,8] = final_bacc_gmean
    final_bacc_bacc = np.nanmean(bacc_bacc)
    res.iloc[5,8] = final_bacc_bacc


    res.to_csv("10CV_MLP-15_Datsets.csv", mode = 'a', float_format='%.4g')

+++++++++++++++++++++++++++++++++++ 1th Dataset +++++++++++++++++++++++++++++++++++
<Original Class>
 pathology_cp_data
2    160
1     90
Name: count, dtype: int64
<Modified Class>
 pathology_cp_data
0    160
1     90
Name: count, dtype: int64
<Imabalance ratio>
  1.78:1
################################################## 1th CV ##################################################
<Class Distribution>
 Train 0:1
 144 81 
 Test 0:1
 16 9


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


------------------------ Delta Rule ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
------------------------ Logistic Regression ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
------------------------ Our Method (Accuracy) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
------------------------ Our Method (F1) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
------------------------ Our Method (F0.5) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
------------------------ Our Method (F2) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
------------------------ Our Method (Gmean) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
------------------------ Our Method (B_Accuracy) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
################################################## 2th CV ##################################################
<Class Distribution>
 Train 0:1
 144 81 


<ipython-input-11-7d975422479e>:295: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.696' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res.iloc[0,1] = final_ada_acc
<ipython-input-11-7d975422479e>:308: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.712' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res.iloc[0,2] = final_bce_acc
<ipython-input-11-7d975422479e>:321: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6960000000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  res.iloc[0,3] = final_acc_acc
<ipython-input-11-7d975422479e>:334: FutureWarning: Setting an item of incompatible dtype is deprecated and wil

------------------------ Delta Rule ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
------------------------ Logistic Regression ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
------------------------ Our Method (Accuracy) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
------------------------ Our Method (F1) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
------------------------ Our Method (F0.5) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
------------------------ Our Method (F2) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
------------------------ Our Method (Gmean) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
------------------------ Our Method (B_Accuracy) ------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
################################################## 2th CV ##################################################
<Class Distribution>
 Train 0:1
 139 8

...

------------------------ Delta Rule ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
------------------------ Logistic Regression ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
------------------------ Our Method (Accuracy) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
------------------------ Our Method (F1) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
------------------------ Our Method (F0.5) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
------------------------ Our Method (F2) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
------------------------ Our Method (Gmean) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
------------------------ Our Method (B_Accuracy) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
################################################## 2th CV ##################################################
<Class Distribution>
 Train 0:1
 250 76

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


------------------------ Delta Rule ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
------------------------ Logistic Regression ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
------------------------ Our Method (Accuracy) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
------------------------ Our Method (F1) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
------------------------ Our Method (F0.5) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
------------------------ Our Method (F2) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
------------------------ Our Method (Gmean) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
------------------------ Our Method (B_Accuracy) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
################################################## 2th CV ##################################################
<Class Distribution>
 Train 0:1
 300 28

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


------------------------ Delta Rule ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
------------------------ Logistic Regression ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
------------------------ Our Method (Accuracy) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
------------------------ Our Method (F1) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
------------------------ Our Method (F0.5) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
------------------------ Our Method (F2) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
------------------------ Our Method (Gmean) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
------------------------ Our Method (B_Accuracy) ------------------------
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
################################################## 2th CV ##################################################
<Class Distribution>
 Train 0:1
 290 53

In [12]:
res

,Dataset,Delta,BCE,AccAim,F1Aim,F.5Aim,F2Aim,GAim,BAim
Acc,D,0.86867,0.85579,0.86604,0.85322,0.86092,0.85835,0.85330,0.85849
F1,S,0.21428,0.16904,0.18571,0.16648,0.17571,0.23220,0.24740,0.27856
F.5,15,0.34286,0.25476,0.29286,0.25084,0.26688,0.32871,0.34121,0.38226
F2,C,0.15692,0.13179,0.13692,0.13072,0.13417,0.20335,0.21891,0.24356
Gmean,S,0.27876,0.23432,0.23794,0.23338,0.23616,0.31368,0.32693,0.37277
B_Acc,V,0.56666,0.55227,0.55833,0.55075,0.55530,0.58953,0.59499,0.60650
